In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy as np
import pickle
from glob import glob

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

> # Part 1: Task Description and Exploratory Analysis
> # ? Not sure to use one file, or all in training files

In [2]:
import pickle

# select one of the file from train dataset to do data exploriation 
data = 0
with open('../input/kaggle-competitions-download-c-cse151bspring/new_train/new_train/0.pkl', 'rb') as f:
    data = pickle.load(f)

### Position Input for all Agents

In [3]:
x_position_in = np.reshape(data['p_in'][:,:,0], 60*19)
y_position_in = np.reshape(data['p_in'][:,:,1], 60*19)

plt.hist(x_position_in[x_position_in != 0])
plt.xlabel('x axis position')
plt.ylabel('Frequency')
plt.title('X axis Input Position for all agents')

NameError: name 'plt' is not defined

In [4]:
plt.hist(y_position_in[x_position_in != 0])
plt.xlabel('y axis position')
plt.ylabel('Frequency')
plt.title('Y axis Input Position for all agents')

NameError: name 'plt' is not defined

## Position Output for all agents

In [5]:
x_position_out = np.reshape(data['p_out'][:,:,0], 60*30)
y_position_out = np.reshape(data['p_out'][:,:,1], 60*30)

plt.hist(x_position_out[x_position_out != 0])
plt.xlabel('x axis position')
plt.ylabel('Frequency')
plt.title('X axis Output Position for all agents')

NameError: name 'plt' is not defined

In [6]:
plt.hist(y_position_out[y_position_out != 0])
plt.xlabel('y axis position')
plt.ylabel('Frequency')
plt.title('Y axis Output Position for all agents')

NameError: name 'plt' is not defined

## Velocity Input for all agents

In [7]:
x_velocity_in = np.reshape(data['v_in'][:,:,0], 60*19)
y_velocity_in = np.reshape(data['v_in'][:,:,1], 60*19)

plt.hist(x_velocity_in[x_velocity_in != 0])
plt.xlabel('x axis position')
plt.ylabel('Frequency')
plt.title('X axis Input Velocity for all agents')

NameError: name 'plt' is not defined

In [8]:
plt.hist(y_velocity_in[y_velocity_in != 0])
plt.xlabel('y axis position')
plt.ylabel('Frequency')
plt.title('Y axis Input Velocity for all agents')

NameError: name 'plt' is not defined

## Velocity Output for all agents

In [9]:
x_velocity_out = np.reshape(data['v_out'][:,:,0], 60*30)
y_velocity_out = np.reshape(data['v_out'][:,:,1], 60*30)

plt.hist(x_velocity_in[x_velocity_in != 0])
plt.xlabel('y axis position')
plt.ylabel('Frequency')
plt.title('Y axis Input Velocity for all agents')

NameError: name 'plt' is not defined

# Data Processing

## Load data for training and testing data

In [10]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_data  = ArgoverseDataset(data_path="../input/kaggle-competitions-download-c-cse151bspring/new_train/new_train/")
test_data  = ArgoverseDataset(data_path="../input/kaggle-competitions-download-c-cse151bspring/new_val_in/new_val_in/")

In [11]:
import itertools
from torch.utils.data import Dataset, DataLoader

batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.transpose(np.dstack([scene['p_in'][:,:,0].reshape(60*19),scene['p_in'][:,:,1].reshape(60*19),scene['v_in'][:,:,0].reshape(60*19),scene['v_in'][:,:,1].reshape(60*19)]).reshape(60*19,4)) for scene in batch]
    
    indexs = [np.where(scene['agent_id'] == scene['track_id'][:,0].reshape(60))[0][0] for scene in batch]
    p_out = [scene['p_out'][index] for (scene,index) in zip(batch, indexs)]
    v_out = [scene['v_out'][index] for (scene,index) in zip(batch, indexs)]
    out = [np.concatenate((p[:,0],p[:,1],v[:,0],v[:,1])) for(p,v) in zip(p_out,v_out)]
    
    
    #out = [np.concatenate((scene['p_out'][:,:,0].reshape(60*30), scene['p_out'][:,:,1].reshape(60*30))) for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    #print(inp.type())
    #print(out.type())
    
    return [inp, out]

train_loader = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [12]:
_, (example_datas, labels) = next(enumerate(train_loader))

"""
For input:
1st dimension is batch size = 4
2nd dimension is channel size = 4 (p_in_x, p_in_y, v_in_x, v_in_y)
3rd dimension is # sample 60 (# agents) x 19 (timestemps)

For output:
1st dimension is batch size = 4
2nd dimension is final ouput of the agent 
0_29: p_out_x, 
30_59: p_out_y,
60-89: v_out_x, 
100:119: v_out_y
"""
print(np.shape(example_datas))
print(np.shape(labels))
example_datas

torch.Size([4, 4, 1140])
torch.Size([4, 120])


tensor([[[3277, 3277, 3277,  ...,    0,    0,    0],
         [1947, 1947, 1947,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]],

        [[ 823,  823,  823,  ...,    0,    0,    0],
         [2068, 2068, 2068,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]],

        [[3022, 3022, 3021,  ...,    0,    0,    0],
         [1595, 1594, 1593,  ...,    0,    0,    0],
         [  -8,   -7,   -7,  ...,    0,    0,    0],
         [  -7,   -7,   -6,  ...,    0,    0,    0]],

        [[ 152,  152,  151,  ...,    0,    0,    0],
         [3732, 3732, 3732,  ...,    0,    0,    0],
         [  -3,   -4,   -4,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]]])

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

nfilt1 = 10
nfilt2 = 20
filt_size = 10
pool_size1 = 4
pool_size2 = 5
in_length = 1140
out_length = 120

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(4, nfilt1, filt_size)
        self.pool1 = nn.MaxPool1d(pool_size1, ceil_mode=True)
        self.conv2 = nn.Conv2d(nfilt1, nfilt2, filt_size)
        self.pool2 = nn.MaxPool1d(pool_size2, ceil_mode=True)
        self.fc1 = nn.Linear(int(in_length/pool_size1/pool_size2)*nfilt1*nfilt2, 5000)
        self.fc2 = nn.Linear(9001, out_length)
        


    def forward(self, x):
        print("forward x is ",x.type())
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.reshape(x, (-1,nfilt1*nfilt2*(int(in_length/pool_size1/pool_size2))))
        x = F.relu(self.fc1(x))
        x = torch.cat((x,m),dim=1)
        x = self.fc2(x)
        return x
    
net = CNN()

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

forward x is  torch.LongTensor


RuntimeError: expected scalar type Long but found Float

In [16]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

        
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    """TODO:
      Deep learning model
      training routine
    """
    show_sample_batch(sample_batch, agent_id)
    break

NameError: name 'val_loader' is not defined